In [1]:
import argparse
import logging
import os
import math
from dataclasses import dataclass, field
import copy
from typing import Dict, List, Optional, Tuple

import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss, MSELoss
from torch.nn import functional as F
from torch import Tensor

from transformers.models.mbart.modeling_mbart import MBartLearnedPositionalEmbedding
from transformers.models.bart.modeling_bart import BartLearnedPositionalEmbedding
from transformers import MBartForConditionalGeneration, MBartConfig, MBart50Tokenizer
from transformers import PreTrainedTokenizerFast
from transformers.models.bart.modeling_bart import shift_tokens_right
from transformers.models.longformer.modeling_longformer import LongformerSelfAttention
from transformers.models.bart.modeling_bart import BartLearnedPositionalEmbedding

import warnings
warnings.filterwarnings(action='ignore')

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

class LongformerSelfAttentionForMBart(nn.Module):
    def __init__(self, config, layer_id):
        super().__init__()
        self.embed_dim = config.d_model
        self.longformer_self_attn = LongformerSelfAttention(config, layer_id=layer_id)
        self.output = nn.Linear(self.embed_dim, self.embed_dim)

    def forward(
        self,
        hidden_states: torch.Tensor,
        key_value_states: Optional[torch.Tensor] = None,
        past_key_value: Optional[Tuple[torch.Tensor]] = None,
        attention_mask: Optional[torch.Tensor] = None,
        layer_head_mask: Optional[torch.Tensor] = None,
        output_attentions: bool = False,
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor]]]:

        is_cross_attention = key_value_states is not None
        bsz, tgt_len, embed_dim = hidden_states.size()

        attention_mask = attention_mask.squeeze(dim=1)
        attention_mask = attention_mask[:,0]

        is_index_masked = attention_mask < 0
        is_index_global_attn = attention_mask > 0
        is_global_attn = is_index_global_attn.flatten().any().item()

        outputs = self.longformer_self_attn(
            hidden_states,
            attention_mask=attention_mask,
            layer_head_mask=None,
            is_index_masked=is_index_masked,
            is_index_global_attn=is_index_global_attn,
            is_global_attn=is_global_attn,
            output_attentions=output_attentions,
        )

        attn_output = self.output(outputs[0])

        return (attn_output,) + outputs[1:] if len(outputs) == 2 else (attn_output, None, None)

class LongformerEncoderDecoderForConditionalGeneration(MBartForConditionalGeneration):
    def __init__(self, config):
        print(f'config: {config}')
        super().__init__(config)
        
        print(f'before if statement')

        if config.attention_mode == 'n2':
            pass  # do nothing, use BertSelfAttention instead
        else:

            print('instantiating MBartLearnedPositionalEmbedding for encoder...')

            self.model.encoder.embed_positions = MBartLearnedPositionalEmbedding(
                config.max_encoder_position_embeddings,
                config.d_model)

            print('instantiating MBartLearnedPositionalEmbedding for decoder...')
            self.model.decoder.embed_positions = MBartLearnedPositionalEmbedding(
                config.max_decoder_position_embeddings, 
                config.d_model)

            print('replacing attention with long attention...')
            for i, layer in enumerate(self.model.encoder.layers):
                layer.self_attn = LongformerSelfAttentionForMBart(config, layer_id=i)

class LongformerEncoderDecoderConfig(MBartConfig):
    def __init__(self, attention_window: List[int] = None, attention_dilation: List[int] = None,
                 autoregressive: bool = False, attention_mode: str = 'sliding_chunks',
                 gradient_checkpointing: bool = False, **kwargs):
        """
        Args:
            attention_window: list of attention window sizes of length = number of layers.
                window size = number of attention locations on each side.
                For an affective window size of 512, use `attention_window=[256]*num_layers`
                which is 256 on each side.
            attention_dilation: list of attention dilation of length = number of layers.
                attention dilation of `1` means no dilation.
            autoregressive: do autoregressive attention or have attention of both sides
            attention_mode: 'n2' for regular n^2 self-attention, 'tvm' for TVM implemenation of Longformer
                selfattention, 'sliding_chunks' for another implementation of Longformer selfattention
        """
        super().__init__(**kwargs)
        self.attention_window = attention_window
        self.attention_dilation = attention_dilation
        self.autoregressive = autoregressive
        self.attention_mode = attention_mode
        self.gradient_checkpointing = gradient_checkpointing
        assert self.attention_mode in ['tvm', 'sliding_chunks', 'n2']

/home/bcolnar/.miniconda3/envs/diploma/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def create_long_model(save_model_to, base_model, tokenizer_name_or_path, attention_window, max_pos):
    tokenizer = MBart50Tokenizer.from_pretrained(tokenizer_name_or_path, model_max_length=max_pos)
    model = MBartForConditionalGeneration.from_pretrained(base_model)
    config = LongformerEncoderDecoderConfig.from_pretrained(base_model)

    model.config = config

    # in BART attention_probs_dropout_prob is attention_dropout, but LongformerSelfAttention
    # expects attention_probs_dropout_prob, so set it here

    config.attention_probs_dropout_prob = config.attention_dropout
    config.architectures = ['LongformerEncoderDecoderForConditionalGeneration', ]

    # extend position embeddings
    tokenizer.model_max_length = max_pos
    tokenizer.init_kwargs['model_max_length'] = max_pos
    current_max_pos, embed_size = model.model.encoder.embed_positions.weight.shape
    assert current_max_pos == config.max_position_embeddings + 2

    config.max_encoder_position_embeddings = max_pos
    config.max_decoder_position_embeddings = config.max_position_embeddings
    del config.max_position_embeddings
    max_pos += 2  # NOTE: BART has positions 0,1 reserved, so embedding size is max position + 2
    assert max_pos >= current_max_pos

    # allocate a larger position embedding matrix for the encoder
    new_encoder_pos_embed = model.model.encoder.embed_positions.weight.new_empty(max_pos, embed_size)


    print(f'new_encoder_pos_embed: {new_encoder_pos_embed}')
    print(f'new_encoder_pos_embed.shape: {new_encoder_pos_embed.shape}')

    # copy position embeddings over and over to initialize the new position embeddings
    k = 2
    step = current_max_pos - 2
    while k < max_pos - 1:
        new_encoder_pos_embed[k:(
            k + step)] = model.model.encoder.embed_positions.weight[2:]
        k += step
        
    model.model.encoder.embed_positions.weight.data = new_encoder_pos_embed

    # allocate a larger position embedding matrix for the decoder
    # new_decoder_pos_embed = model.model.decoder.embed_positions.weight.new_empty(max_pos, embed_size)
    # # copy position embeddings over and over to initialize the new position embeddings
    # k = 2
    # step = current_max_pos - 2
    # while k < max_pos - 1:
    #     new_decoder_pos_embed[k:(k + step)] = model.model.decoder.embed_positions.weight[2:]
    #     k += step
    # model.model.decoder.embed_positions.weight.data = new_decoder_pos_embed

    # replace the `modeling_bart.SelfAttention` object with `LongformerSelfAttention`

    config.attention_window = [attention_window] * config.num_hidden_layers
    config.attention_dilation = [1] * config.num_hidden_layers

    for i, layer in enumerate(model.model.encoder.layers):
        longformer_self_attn_for_mbart = LongformerSelfAttentionForMBart(config, layer_id=i)

        longformer_self_attn_for_mbart.longformer_self_attn.query = layer.self_attn.q_proj
        longformer_self_attn_for_mbart.longformer_self_attn.key = layer.self_attn.k_proj
        longformer_self_attn_for_mbart.longformer_self_attn.value = layer.self_attn.v_proj

        longformer_self_attn_for_mbart.longformer_self_attn.query_global = copy.deepcopy(
            layer.self_attn.q_proj)
        longformer_self_attn_for_mbart.longformer_self_attn.key_global = copy.deepcopy(
            layer.self_attn.k_proj)
        longformer_self_attn_for_mbart.longformer_self_attn.value_global = copy.deepcopy(
            layer.self_attn.v_proj)

        longformer_self_attn_for_mbart.output = layer.self_attn.out_proj

        layer.self_attn = longformer_self_attn_for_mbart

    logger.info(f'saving model to {save_model_to}')
    model.save_pretrained(save_model_to)
    tokenizer.save_pretrained(save_model_to, None)

    return model, tokenizer


In [3]:
from transformers import AutoTokenizer

save_model_to = './tmp/20k/mbart-long'# './tmp/mbart-long'
base_model = 'facebook/mbart-large-50'
tokenizer_name_or_path = 'facebook/mbart-large-50'
attention_window = 512
max_pos =  20480 # 16384 # mutiple of origin maximum encoder positions ?

print('new model arguments: ')
print(f'save_model_to={save_model_to}, base_model={base_model}, tokenizer_name_or_path={tokenizer_name_or_path}, attention_window={attention_window}, max_pos={max_pos}')
print('creating new model...')

create_long_model(
    save_model_to=save_model_to,
    base_model=base_model,
    tokenizer_name_or_path=tokenizer_name_or_path,
    attention_window=attention_window,
    max_pos=max_pos
)

tokenizer = MBart50Tokenizer.from_pretrained(save_model_to)
model = LongformerEncoderDecoderForConditionalGeneration.from_pretrained(save_model_to)

model.model.encoder.config.gradient_checkpointing = True
model.model.decoder.config.gradient_checkpointing = True

new model arguments: 
save_model_to=./tmp/20k/mbart-long, base_model=facebook/mbart-large-50, tokenizer_name_or_path=facebook/mbart-large-50, attention_window=512, max_pos=20480
creating new model...
new_encoder_pos_embed: tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
new_encoder_pos_embed.shape: torch.Size([20482, 1024])


INFO:__main__:saving model to ./tmp/20k/mbart-long


config: MBartConfig {
  "_name_or_path": "./tmp/20k/mbart-long",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "MBartForConditionalGeneration"
  ],
  "attention_dilation": [
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1
  ],
  "attention_dropout": 0.0,
  "attention_mode": "sliding_chunks",
  "attention_probs_dropout_prob": 0.0,
  "attention_window": [
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "autoregressive": false,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "enco

In [4]:
print(model)

LongformerEncoderDecoderForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(20482, 1024)
      (layers): ModuleList(
        (0): MBartEncoderLayer(
          (self_attn): LongformerSelfAttentionForMBart(
            (longformer_self_attn): LongformerSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (query_global): Linear(in_features=1024, out_features=1024, bias=True)
              (key_global): Linear(in_features=1024, out_features=1024, bias=True)
              (value_global): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (output): Linear(in_featur

In [7]:
max_seq_len = 512 * 32

def summarize(text, max_len):

    context_tokens = ['<s>'] + tokenizer.tokenize(text) + ['</s>']
    input_ids = tokenizer.convert_tokens_to_ids(context_tokens) 

    if len(input_ids) < max_seq_len:   
            while len(input_ids) < max_seq_len: 
                input_ids += [tokenizer.pad_token_id] 
    else:
        input_ids = input_ids[:max_seq_len - 1] + [   
            tokenizer.eos_token_id]

    print('input_ids: ',len(input_ids))


    model.model.encoder.config.gradient_checkpointing = True
    model.model.decoder.config.gradient_checkpointing = True

    res_ids = model.generate(torch.tensor([input_ids]),
                                        max_length=max_len,
                                        num_beams=5,
                                        no_repeat_ngram_size = 3,
                                        eos_token_id=tokenizer.eos_token_id,
                                        bad_words_ids=[[tokenizer.unk_token_id]])        
    res = tokenizer.batch_decode(res_ids.tolist(), skip_special_tokens=True)[0]
    
    return print(res)

summarize("Rimsko cesarstvo (latinsko Imperivm Romanvm, grško Βασιλεία τῶν Ῥωμαίων, Basileía tōn Rhōmaíōn) je bilo obdobje starega Rima, ki je sledilo Rimski republiki. Kot država je obsegalo veliko ozemlje okoli Sredozemskega morja v Evropi, severni Afriki in zahodni Aziji. V cesarstvu so vladali cesarji. Od začetka vladavine cesarja Avgusta do vojaške anarhije v 3. stoletju je bila država principat z Italijo kot metropolo provinc in Rimom kot edino prestolnico (27 pr. n. št. - 286 n. št.). Po krizi 3. stoletja je bilo cesarstvo razdeljeno v Zahodno rimsko cesarstvo in Vzhodno rimsko cesarstvo. Slednje je znano tudi kot Bizantinsko cesarstvo. Cesarstvi sta imeli vsako svojega cesarja. Uradna prestolnica obeh cesarstev je do leta 476 ostal Rim. Tisto leto so Raveno zasedli Odoakerjevi Ostrogoti in odstavili zadnjega zahodnorimskega cesarja Romula Avgusta, zato so cesarske insignije prenesli v Konstantinopel. S sprejetjem krščanstva kot državne vere Rimskega cesarstva leta 380 in padcem Zahodnega rimskega cesarstva se je končalo obdobje klasične antike in začel srednji vek. Ti dogodki in postopna helenizacija Vzhodnega rimskega cesarstva so razlog, da zgodovinarji srednjeveško Rimsko cesarstvo, ki je ostalo v vzhodnih rimskih provincah, imenujejo Bizantinsko cesarstvo. Prvi dve stoletji cesarstva sta bili obdobje stabilnosti in razcveta brez primere, znano kot Pax Romana (rimski mir). V 3. stoletju je cesarstvo doživelo krizo, ki je ogrozila njegov obstoj, saj sta se Galsko in Palmirsko cesarstvo odcepila. Na prestolu se je zvrstila vrsta kratkoživih cesarjev, pogosto legionarjev in pogosto več hkrati. Cesarstvo je ponovno združil Avrelijan (270–275). Dioklecijan je poskusil cesarstvo stabilizirati in ga je leta 286 razdelil na latinski zahod in grški vzhod. V 4. stoletju so se po Milanskem ediktu iz leta 313 začeli na vplivne državne položaje vzpenjati kristjani. Kmalu zatem se je začelo obdobje velikih selitev, obsežnih vpadov germanskih ljudstev in Hunov pod vodstvom Atile, kar je pripeljalo do propada Zahodnega rimskega cesarstva. S padcem Ravene pod germanske Herule in z Odoakerjevo odstavitvijo cesarja Romula Avgusta leta 476 je Zahodno rimsko cesarstvo dokončno propadlo, kar je formalno potrdil vzhodnorimski cesar Zenon leta 480. Nekatere države na ozemljih nekdanjega Zahodnega rimskega cesarstva so se kasneje imele za njegove dediče in so se potegovale za vrhovno oblast rimskih cesarjev. Najpomembnejše med njimi je bilo Sveto rimsko cesarstvo. Vzhodno rimsko cesarstvo je preživelo še celo tisočletje, dokler niso Konstantinopla leta 1453 zavzeli osmanski Turki pod vodstvom sultana Mehmeda II.[op 4] Zaradi velikega ozemlja in dolgega obstoja Rimskega cesarstva so rimske upravne prakse in kultura močno in trajno vplivale na razvoj jezika, religije, umetnosti, arhitekture, filozofije, prava in oblik vladanja ne samo na ozemlju, ki ga je obsegalo, temveč tudi daleč preko meja. Jezik Rimljanov se je razvil v romanske jezike srednjeveškega in modernega sveta, medtem ko je klasična grščina postala jezik Vzhodnega rimskega cesarstva. Sprejetje krščanstva v cesarstvu je privedlo do oblikovanja srednjeveškega krščanstva. Grška in rimska umetnost sta močno vplivali na italijansko renesanso. Rimska arhitekturna tradicija je služila kot osnova za romansko, renesančno in neoklasično arhitekturo, močno pa je vplivala tudi na islamsko arhitekturo. Rimsko pravo ima naslednike v mnogih današnjih pravnih sistemih, na primer v Napoleonovem zakoniku, pa tudi v francoskem in italijanskem sodobnem pravu; rimske republiške institucije so pustile trajno zapuščino, ki je vplivala na srednjeveške italijanske mestne republike in države, pozneje pa tudi na politično ureditev v ZDA in drugih modernih ", 512)

input_ids:  16384
Rimsko cesarstvo (latinsko Imperivm Romanvm, grško Βασιλεία τῶν ὄωμαίων, Basileía tōn Rhōmaíōn) je bilo obdobje starega Rima, ki je sledilo Rimski republiki. Kot država je obsegalo veliko ozemlje okoli Sredozemskega morja v Evropi, severni Afriki in zahodni Aziji. V cesarstvu so vladali cesarji. Od začetka vladavine cesarja Avgusta do vojaške anarhije v 3. stoletju je bila država principat z Italijo kot metropolo provinc in Rimom kot edino prestolnico (27 pr. n. št. - 286 n. m.). Po krizi 3. stoletja je bilo cesarstvo razdeljeno v Zahodno rimske cesarstvo in Vzhodno rimsko cearstvo. Slednje je znano tudi kot Bizantinsko cesarstvo. Cesarstvi sta imeli vsako svojega cesarja. Uradna prestolnica obeh cesarstev je do leta 476 ostal Rim. Tisto leto so Raveno zasedli Odoakerjevi Ostrogoti in odstavili zadnjega zahodnorimskega cesarja Romula Avgust.
